In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1. Preprocessing
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # grayscale
])

# 2. Load dataset
dataset = datasets.ImageFolder(root="/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset",
                               transform=transform)

# 3. Create DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# 4. Check classes
print(dataset.classes)
# Output: ['COVID', 'Lung_Opacity', 'Normal', 'Viral_Pneumonia']


['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']


In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import timm  # for EfficientNet
from sklearn.metrics import classification_report, confusion_matrix


In [7]:
# Transformations (data augmentation + normalization)
transform = transforms.Compose([
    transforms.Resize((224, 224)),        # EfficientNet input size
    transforms.RandomHorizontalFlip(),    # Augmentation
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])   # X-rays are grayscale
])

# Dataset path
data_dir = "/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset"  # replace with your path

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Train / validation split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Check classes
print(dataset.classes)
# ['COVID', 'Lung_Opacity', 'Normal', 'Viral_Pneumonia']


['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']


In [8]:
# Load pretrained EfficientNet (B0) from timm
model = timm.create_model('efficientnet_b0', pretrained=True)

# Modify classifier for 4 classes
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 4)  # 4 classes
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [10]:
epochs = 10  # you can increase later

for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")


Epoch [1/10], Loss: 0.5572, Train Acc: 78.69%
Epoch [2/10], Loss: 0.4197, Train Acc: 83.92%
Epoch [3/10], Loss: 0.3911, Train Acc: 85.08%
Epoch [4/10], Loss: 0.3630, Train Acc: 86.06%
Epoch [5/10], Loss: 0.3460, Train Acc: 86.61%
Epoch [6/10], Loss: 0.3290, Train Acc: 87.17%
Epoch [7/10], Loss: 0.3135, Train Acc: 88.06%
Epoch [8/10], Loss: 0.3017, Train Acc: 88.33%
Epoch [9/10], Loss: 0.2880, Train Acc: 88.89%
Epoch [10/10], Loss: 0.2754, Train Acc: 89.20%


In [11]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Classification Report:\n")
print(classification_report(all_labels, all_preds, target_names=dataset.classes))

print("Confusion Matrix:\n")
print(confusion_matrix(all_labels, all_preds))


Classification Report:

                 precision    recall  f1-score   support

          COVID       0.86      0.75      0.80      1465
   Lung_Opacity       0.84      0.84      0.84      2382
         Normal       0.88      0.92      0.90      4087
Viral Pneumonia       0.94      0.94      0.94       532

       accuracy                           0.87      8466
      macro avg       0.88      0.86      0.87      8466
   weighted avg       0.87      0.87      0.87      8466

Confusion Matrix:

[[1095  169  200    1]
 [  84 2008  285    5]
 [  91  205 3763   28]
 [   3    6   21  502]]


In [12]:
torch.save(model.state_dict(), "efficientnet_xray.pth")
